In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, Text
from dfply import *

%matplotlib inline

In [2]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

2021-10-29 14:09:33.971514: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Retrieval - matrix factorisation tower

In [3]:
ratings

<PrefetchDataset shapes: {bucketized_user_age: (), movie_genres: (None,), movie_id: (), movie_title: (), raw_user_age: (), timestamp: (), user_gender: (), user_id: (), user_occupation_label: (), user_occupation_text: (), user_rating: (), user_zip_code: ()}, types: {bucketized_user_age: tf.float32, movie_genres: tf.int64, movie_id: tf.string, movie_title: tf.string, raw_user_age: tf.float32, timestamp: tf.int64, user_gender: tf.bool, user_id: tf.string, user_occupation_label: tf.int64, user_occupation_text: tf.string, user_rating: tf.float32, user_zip_code: tf.string}>

In [4]:
ratings_reduced = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_occupation":x["user_occupation_text"],
})
movies_reduced = movies.map(lambda x: x["movie_title"])

In [5]:
ratings_reduced

<MapDataset shapes: {movie_title: (), user_id: (), user_occupation: ()}, types: {movie_title: tf.string, user_id: tf.string, user_occupation: tf.string}>

In [6]:
tf.random.set_seed(42)
shuffled = ratings_reduced.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [7]:
movie_titles = movies_reduced.batch(1_000)
user_ids = ratings_reduced.batch(1_000_000).map(lambda x: x["user_id"])
user_occupation = ratings_reduced.batch(1_000_000).map(lambda x: x["user_occupation"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))
unique_user_occupations = np.unique(np.concatenate(list(user_occupation)))

unique_movie_titles[:10]

2021-10-29 14:09:34.291270: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

In [8]:
embedding_dimension = 32

In [9]:
user_model = tf.keras.Sequential([
    #user id feature
    tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
    tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [10]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [11]:
import tensorflow_recommenders as tfrs
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies_reduced.batch(128).map(movie_model)
)

In [12]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [13]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

In [14]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [15]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [16]:
model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 8s 595ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0013 - factorized_top_k/top_5_categorical_accuracy: 0.0099 - factorized_top_k/top_10_categorical_accuracy: 0.0205 - factorized_top_k/top_50_categorical_accuracy: 0.1005 - factorized_top_k/top_100_categorical_accuracy: 0.1774 - loss: 69885.1129 - regularization_loss: 0.0000e+00 - total_loss: 69885.1129
Epoch 2/3
10/10 [==============================] - 6s 616ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0028 - factorized_top_k/top_5_categorical_accuracy: 0.0188 - factorized_top_k/top_10_categorical_accuracy: 0.0380 - factorized_top_k/top_50_categorical_accuracy: 0.1696 - factorized_top_k/top_100_categorical_accuracy: 0.2931 - loss: 67523.3707 - regularization_loss: 0.0000e+00 - total_loss: 67523.3707
Epoch 3/3
10/10 [==============================] - 6s 639ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0035 - factorized_top_k/top_5_categorical_accuracy: 

In [17]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 3s 329ms/step - factorized_top_k/top_1_categorical_accuracy: 9.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0096 - factorized_top_k/top_10_categorical_accuracy: 0.0224 - factorized_top_k/top_50_categorical_accuracy: 0.1250 - factorized_top_k/top_100_categorical_accuracy: 0.2327 - loss: 31079.0635 - regularization_loss: 0.0000e+00 - total_loss: 31079.0635


{'factorized_top_k/top_1_categorical_accuracy': 0.0008999999845400453,
 'factorized_top_k/top_5_categorical_accuracy': 0.009600000455975533,
 'factorized_top_k/top_10_categorical_accuracy': 0.022350000217556953,
 'factorized_top_k/top_50_categorical_accuracy': 0.125,
 'factorized_top_k/top_100_categorical_accuracy': 0.23270000517368317,
 'loss': 28244.771484375,
 'regularization_loss': 0,
 'total_loss': 28244.771484375}

In [18]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies_reduced.batch(100), movies_reduced.batch(100).map(model.movie_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

Recommendations for user 42: [b'Bridges of Madison County, The (1995)'
 b'Father of the Bride Part II (1995)' b'Rudy (1993)']


Investgate adding user occupation

In [19]:
class UserModel(tf.keras.Model):

  def __init__(self, use_occupation):
    super().__init__()

    self._use_occupation = use_occupation

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])

    if use_occupation:
        self.user_embedding2 = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_occupations, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_occupations) + 1, 32),
    ])


  def call(self, inputs):
    if not self._use_occupation:
      return self.user_embedding(inputs["user_id"])

    return tf.concat([
        self.user_embedding(inputs["user_id"]),
        self.user_embedding2(inputs["user_occupation"])
    ],1)

In [20]:
class MovieModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, 32)
    ])

  def call(self, titles):
    return self.title_embedding(titles)

In [26]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, use_occupations):
    super().__init__()
    self.query_model = tf.keras.Sequential([
      UserModel(use_occupations),
      tf.keras.layers.Dense(32)
    ])
    self.candidate_model = tf.keras.Sequential([
      MovieModel(),
      tf.keras.layers.Dense(32)
    ])
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies_reduced.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"],
        "user_occupation": features["user_occupation"],
    })
    movie_embeddings = self.candidate_model(features["movie_title"])

    return self.task(query_embeddings, movie_embeddings)

In [22]:
tf.random.set_seed(42)
shuffled = ratings_reduced.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

cached_train = train.shuffle(100_000).batch(2048)
cached_test = test.batch(4096).cache()

In [27]:
model = MovielensModel(use_occupations=False)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

Epoch 1/3
Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


40/40 [==============================] - 9s 182ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0203 - factorized_top_k/top_5_categorical_accuracy: 0.0328 - factorized_top_k/top_10_categorical_accuracy: 0.0430 - factorized_top_k/top_50_categorical_accuracy: 0.0988 - factorized_top_k/top_100_categorical_accuracy: 0.1599 - loss: 14667.0136 - regularization_loss: 0.0000e+00 - total_loss: 14667.0136
Epoch 2/3
40/40 [==============================] - 8s 177ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0028 - factorized_top_k/top_5_categorical_accuracy: 0.0130 - factorized_top_k/top_10_categorical_accuracy: 0.0256 - factorized_top_k/top_50_categorical_accuracy: 0.1127 - factorized_top_k/top_100_categorical_accuracy: 0.2110 - loss: 14185.7672 - regularization_loss: 0.0000e+00 - total_loss: 14185.7672
Epoch 3/3
40/40 [==============================] - 8s 183ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0020 - factorized_top_k/top_5_categorical_accuracy: 0.0146 - f

Consider rewriting this model with the Functional API.


5/5 [==============================] - 2s 299ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0011 - factorized_top_k/top_5_categorical_accuracy: 0.0084 - factorized_top_k/top_10_categorical_accuracy: 0.0203 - factorized_top_k/top_50_categorical_accuracy: 0.1138 - factorized_top_k/top_100_categorical_accuracy: 0.2224 - loss: 31038.6195 - regularization_loss: 0.0000e+00 - total_loss: 31038.6195
Top-100 accuracy (train): 0.29.
Top-100 accuracy (test): 0.22.


In [28]:
model = MovielensModel(use_occupations=True)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

Epoch 1/3
Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


40/40 [==============================] - 9s 181ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0111 - factorized_top_k/top_5_categorical_accuracy: 0.0228 - factorized_top_k/top_10_categorical_accuracy: 0.0331 - factorized_top_k/top_50_categorical_accuracy: 0.0905 - factorized_top_k/top_100_categorical_accuracy: 0.1535 - loss: 14642.0935 - regularization_loss: 0.0000e+00 - total_loss: 14642.0935
Epoch 2/3
40/40 [==============================] - 7s 160ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0029 - factorized_top_k/top_5_categorical_accuracy: 0.0140 - factorized_top_k/top_10_categorical_accuracy: 0.0270 - factorized_top_k/top_50_categorical_accuracy: 0.1154 - factorized_top_k/top_100_categorical_accuracy: 0.2116 - loss: 14185.8808 - regularization_loss: 0.0000e+00 - total_loss: 14185.8808
Epoch 3/3
40/40 [==============================] - 7s 156ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0023 - factorized_top_k/top_5_categorical_accuracy: 0.0142 - f

Consider rewriting this model with the Functional API.


5/5 [==============================] - 2s 295ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0012 - factorized_top_k/top_5_categorical_accuracy: 0.0086 - factorized_top_k/top_10_categorical_accuracy: 0.0197 - factorized_top_k/top_50_categorical_accuracy: 0.1134 - factorized_top_k/top_100_categorical_accuracy: 0.2189 - loss: 31045.4759 - regularization_loss: 0.0000e+00 - total_loss: 31045.4759
Top-100 accuracy (train): 0.30.
Top-100 accuracy (test): 0.22.


Investigate time impact

In [29]:
ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "timestamp": x["timestamp"],
})
movies = movies.map(lambda x: x["movie_title"])

In [30]:
timestamps = np.concatenate(list(ratings.map(lambda x: x["timestamp"]).batch(100)))

max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)

unique_movie_titles = np.unique(np.concatenate(list(movies.batch(1000))))
unique_user_ids = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["user_id"]))))

In [31]:
class UserModel(tf.keras.Model):

  def __init__(self, use_timestamps):
    super().__init__()

    self._use_timestamps = use_timestamps

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])

    if use_timestamps:
      self.timestamp_embedding = tf.keras.Sequential([
          tf.keras.layers.Discretization(timestamp_buckets.tolist()),
          tf.keras.layers.Embedding(len(timestamp_buckets) + 1, 32),
      ])
      self.normalized_timestamp = tf.keras.layers.Normalization(
          axis=None
      )

      self.normalized_timestamp.adapt(timestamps)

  def call(self, inputs):
    if not self._use_timestamps:
      return self.user_embedding(inputs["user_id"])

    return tf.concat([
        self.user_embedding(inputs["user_id"]),
        self.timestamp_embedding(inputs["timestamp"]),
        tf.reshape(self.normalized_timestamp(inputs["timestamp"]), (-1, 1)),
    ], axis=1)

In [32]:
class MovieModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, 32)
    ])

    self.title_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.title_text_embedding = tf.keras.Sequential([
      self.title_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.title_vectorizer.adapt(movies)

  def call(self, titles):
    return tf.concat([
        self.title_embedding(titles),
        self.title_text_embedding(titles),
    ], axis=1)

In [36]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, use_timestamps):
    super().__init__()
    self.query_model = tf.keras.Sequential([
      UserModel(use_timestamps),
      tf.keras.layers.Dense(32)
    ])
    self.candidate_model = tf.keras.Sequential([
      MovieModel(),
      tf.keras.layers.Dense(32)
    ])
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"],
        "timestamp": features["timestamp"],
    })
    movie_embeddings = self.candidate_model(features["movie_title"])

    return self.task(query_embeddings, movie_embeddings)

In [37]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

cached_train = train.shuffle(100_000).batch(2048)
cached_test = test.batch(4096).cache()

In [38]:
model = MovielensModel(use_timestamps=True)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

Epoch 1/3
Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


40/40 [==============================] - 10s 222ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0089 - factorized_top_k/top_5_categorical_accuracy: 0.0206 - factorized_top_k/top_10_categorical_accuracy: 0.0312 - factorized_top_k/top_50_categorical_accuracy: 0.0941 - factorized_top_k/top_100_categorical_accuracy: 0.1611 - loss: 14545.1347 - regularization_loss: 0.0000e+00 - total_loss: 14545.1347
Epoch 2/3
40/40 [==============================] - 8s 190ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0036 - factorized_top_k/top_5_categorical_accuracy: 0.0180 - factorized_top_k/top_10_categorical_accuracy: 0.0335 - factorized_top_k/top_50_categorical_accuracy: 0.1400 - factorized_top_k/top_100_categorical_accuracy: 0.2566 - loss: 13948.6204 - regularization_loss: 0.0000e+00 - total_loss: 13948.6204
Epoch 3/3
40/40 [==============================] - 8s 192ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0025 - factorized_top_k/top_5_categorical_accuracy: 0.0198 - 

Consider rewriting this model with the Functional API.


5/5 [==============================] - 2s 328ms/step - factorized_top_k/top_1_categorical_accuracy: 9.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0099 - factorized_top_k/top_10_categorical_accuracy: 0.0218 - factorized_top_k/top_50_categorical_accuracy: 0.1233 - factorized_top_k/top_100_categorical_accuracy: 0.2392 - loss: 30681.9876 - regularization_loss: 0.0000e+00 - total_loss: 30681.9876
Top-100 accuracy (train): 0.36.
Top-100 accuracy (test): 0.24.


In [40]:
index = tfrs.layers.factorized_top_k.BruteForce(model.query_model)

index.index_from_dataset(

tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.candidate_model)))

)

query = dict(pd.DataFrame({"user_id": ["42"], "timestamp":[879024327]}).iloc[0].map(lambda x: tf.expand_dims(x,axis=0)))

_, titles = index(query)

print(f"Recommendations for user 42: {titles[0, :5]}")

Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Recommendations for user 42: [b'Flipper (1996)' b'Kazaam (1996)' b'101 Dalmatians (1996)'
 b'Grumpier Old Men (1995)' b'Jack (1996)']


Ranking Recommendation

In [41]:
ratings = tfds.load("movielens/100k-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"],
})

In [42]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [43]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])


unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [44]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [45]:
RankingModel()((["42"],["One Flew Over the Cuckoo's Nest (1975)"]))

Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.03740937]], dtype=float32)>

In [46]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [47]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [48]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [49]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [50]:
model.fit(cached_train, epochs=3,verbose=0)

In [51]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 12ms/step - root_mean_squared_error: 1.1108 - loss: 1.2287 - regularization_loss: 0.0000e+00 - total_loss: 1.2287


{'root_mean_squared_error': 1.1108062267303467,
 'loss': 1.2062578201293945,
 'regularization_loss': 0,
 'total_loss': 1.2062578201293945}

In [52]:
test_ratings = {}
test_movie_titles = ["M*A*S*H (1970)", "Dances with Wolves (1990)", "Speed (1994)","Clear and Present Danger (1994)"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": np.array(["42"]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
M*A*S*H (1970): [[3.5847075]]
Dances with Wolves (1990): [[3.5515532]]
Speed (1994): [[3.5215847]]
Clear and Present Danger (1994): [[3.4958954]]


Rating with user age included

In [53]:
ratings = tfds.load("movielens/100k-ratings", split="train")
ratings

<PrefetchDataset shapes: {bucketized_user_age: (), movie_genres: (None,), movie_id: (), movie_title: (), raw_user_age: (), timestamp: (), user_gender: (), user_id: (), user_occupation_label: (), user_occupation_text: (), user_rating: (), user_zip_code: ()}, types: {bucketized_user_age: tf.float32, movie_genres: tf.int64, movie_id: tf.string, movie_title: tf.string, raw_user_age: tf.float32, timestamp: tf.int64, user_gender: tf.bool, user_id: tf.string, user_occupation_label: tf.int64, user_occupation_text: tf.string, user_rating: tf.float32, user_zip_code: tf.string}>

In [54]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"],
    "user_occupation": x["user_occupation_text"],
    "user_zipcode": x["user_zip_code"],
    "user_age": x["raw_user_age"]
})

In [55]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

Ranking Recommendation

In [168]:
ratings = tfds.load("movielens/100k-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"],
})

In [169]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [170]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])


unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [171]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [172]:
RankingModel()((["42"],["One Flew Over the Cuckoo's Nest (1975)"]))

Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.
2021-10-20 09:48:03.932679: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2021-10-20 09:48:04.143068: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.03740937]], dtype=float32)>

In [173]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [174]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [175]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [176]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [177]:
model.fit(cached_train, epochs=3,verbose=0)

In [178]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 15ms/step - root_mean_squared_error: 1.1108 - loss: 1.2287 - regularization_loss: 0.0000e+00 - total_loss: 1.2287


{'root_mean_squared_error': 1.1108062267303467,
 'loss': 1.2062578201293945,
 'regularization_loss': 0,
 'total_loss': 1.2062578201293945}

In [179]:
test_ratings = {}
test_movie_titles = ["M*A*S*H (1970)", "Dances with Wolves (1990)", "Speed (1994)","Clear and Present Danger (1994)"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": np.array(["42"]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
M*A*S*H (1970): [[3.5847075]]
Dances with Wolves (1990): [[3.5515532]]
Speed (1994): [[3.5215847]]
Clear and Present Danger (1994): [[3.4958954]]


Rating with user age included

In [206]:
ratings = tfds.load("movielens/100k-ratings", split="train")
ratings

<PrefetchDataset shapes: {bucketized_user_age: (), movie_genres: (None,), movie_id: (), movie_title: (), raw_user_age: (), timestamp: (), user_gender: (), user_id: (), user_occupation_label: (), user_occupation_text: (), user_rating: (), user_zip_code: ()}, types: {bucketized_user_age: tf.float32, movie_genres: tf.int64, movie_id: tf.string, movie_title: tf.string, raw_user_age: tf.float32, timestamp: tf.int64, user_gender: tf.bool, user_id: tf.string, user_occupation_label: tf.int64, user_occupation_text: tf.string, user_rating: tf.float32, user_zip_code: tf.string}>

In [207]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"],
    "user_occupation": x["user_occupation_text"],
    "user_zipcode": x["user_zip_code"],
    "user_age": x["raw_user_age"]
})

In [208]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [209]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])
user_occupation = ratings.batch(1_000_000).map(lambda x: x["user_occupation"])
user_zipcode = ratings.batch(1_000_000).map(lambda x: x["user_zipcode"])
user_age = ratings.batch(1_000_000).map(lambda x: x["user_age"])


unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))
unique_user_occupations = np.unique(np.concatenate(list(user_occupation)))
unique_user_zipcode = np.unique(np.concatenate(list(user_zipcode)))
unique_user_age = np.unique(np.concatenate(list(user_age)))

In [214]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute occupation embeddings for users.
    self.user_occ_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_occupations, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_occupations) + 1, embedding_dimension)
    ])

    # Compute Zip embeddings for users.
    self.user_zip_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_zipcode, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_zipcode) + 1, embedding_dimension)
    ])

    #Compute age.
    self.normalised_age = tf.keras.layers.Normalization(axis=None)
    self.normalised_age.adapt(user_age)

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, user_occupation,user_zipcode, user_age, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)
    user_occ_embeddings = self.user_occ_embeddings(user_occupation)
    user_zip_embeddings = self.user_zip_embeddings(user_zipcode)
    normalised_age = tf.reshape(self.normalised_age(user_age),(-1,1))


    return self.ratings(tf.concat([user_embedding, user_occ_embeddings, user_zip_embeddings, normalised_age, movie_embedding], axis=1))

In [211]:
import pprint
for x in ratings.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'user_age': 46.0,
 'user_id': b'138',
 'user_occupation': b'doctor',
 'user_rating': 4.0,
 'user_zipcode': b'53211'}


In [212]:
for x in ratings.filter(lambda x: x['user_id']=="42").as_numpy_iterator():
    pprint.pprint(x)

{'movie_title': b'Clear and Present Danger (1994)',
 'user_age': 30.0,
 'user_id': b'42',
 'user_occupation': b'administrator',
 'user_rating': 5.0,
 'user_zipcode': b'17870'}
{'movie_title': b'Grease (1978)',
 'user_age': 30.0,
 'user_id': b'42',
 'user_occupation': b'administrator',
 'user_rating': 2.0,
 'user_zipcode': b'17870'}
{'movie_title': b'Jumanji (1995)',
 'user_age': 30.0,
 'user_id': b'42',
 'user_occupation': b'administrator',
 'user_rating': 4.0,
 'user_zipcode': b'17870'}
{'movie_title': b'While You Were Sleeping (1995)',
 'user_age': 30.0,
 'user_id': b'42',
 'user_occupation': b'administrator',
 'user_rating': 4.0,
 'user_zipcode': b'17870'}
{'movie_title': b'Two if by Sea (1996)',
 'user_age': 30.0,
 'user_id': b'42',
 'user_occupation': b'administrator',
 'user_rating': 3.0,
 'user_zipcode': b'17870'}
{'movie_title': b'Indiana Jones and the Last Crusade (1989)',
 'user_age': 30.0,
 'user_id': b'42',
 'user_occupation': b'administrator',
 'user_rating': 5.0,
 'user_z

In [215]:
RankingModel()((["42"],["administrator"],["17870"],[30.0],["One Flew Over the Cuckoo's Nest (1975)"]))

Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.01861795]], dtype=float32)>

In [216]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [217]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"],features["user_occupation"], features["user_zipcode"],features["user_age"],features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [218]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

model.fit(cached_train, epochs=3,verbose=0)

model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 17ms/step - root_mean_squared_error: 1.1077 - loss: 1.2219 - regularization_loss: 0.0000e+00 - total_loss: 1.2219


{'root_mean_squared_error': 1.1076884269714355,
 'loss': 1.199728012084961,
 'regularization_loss': 0,
 'total_loss': 1.199728012084961}

In [220]:
test_ratings = {}
test_movie_titles = ["M*A*S*H (1970)", "Dances with Wolves (1990)", "Speed (1994)","Clear and Present Danger (1994)"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": np.array(["42"]),
      "user_occupation": np.array(["administrator"]),
      "user_zipcode": np.array(["17870"]),
      "user_age": np.array([30.0]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
M*A*S*H (1970): [[3.6255841]]
Clear and Present Danger (1994): [[3.620366]]
Dances with Wolves (1990): [[3.5949826]]
Speed (1994): [[3.59139]]


In [56]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])
user_occupation = ratings.batch(1_000_000).map(lambda x: x["user_occupation"])
user_zipcode = ratings.batch(1_000_000).map(lambda x: x["user_zipcode"])
user_age = ratings.batch(1_000_000).map(lambda x: x["user_age"])


unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))
unique_user_occupations = np.unique(np.concatenate(list(user_occupation)))
unique_user_zipcode = np.unique(np.concatenate(list(user_zipcode)))
unique_user_age = np.unique(np.concatenate(list(user_age)))

In [57]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute occupation embeddings for users.
    self.user_occ_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_occupations, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_occupations) + 1, embedding_dimension)
    ])

    # Compute Zip embeddings for users.
    self.user_zip_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_zipcode, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_zipcode) + 1, embedding_dimension)
    ])

    #Compute age.
    self.normalised_age = tf.keras.layers.Normalization(axis=None)
    self.normalised_age.adapt(user_age)

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, user_occupation,user_zipcode, user_age, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)
    user_occ_embeddings = self.user_occ_embeddings(user_occupation)
    user_zip_embeddings = self.user_zip_embeddings(user_zipcode)
    normalised_age = tf.reshape(self.normalised_age(user_age),(-1,1))


    return self.ratings(tf.concat([user_embedding, user_occ_embeddings, user_zip_embeddings, normalised_age, movie_embedding], axis=1))

In [58]:
import pprint
for x in ratings.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'user_age': 46.0,
 'user_id': b'138',
 'user_occupation': b'doctor',
 'user_rating': 4.0,
 'user_zipcode': b'53211'}


In [59]:
for x in ratings.filter(lambda x: x['user_id']=="42").as_numpy_iterator():
    pprint.pprint(x)

{'movie_title': b'Clear and Present Danger (1994)',
 'user_age': 30.0,
 'user_id': b'42',
 'user_occupation': b'administrator',
 'user_rating': 5.0,
 'user_zipcode': b'17870'}
{'movie_title': b'Grease (1978)',
 'user_age': 30.0,
 'user_id': b'42',
 'user_occupation': b'administrator',
 'user_rating': 2.0,
 'user_zipcode': b'17870'}
{'movie_title': b'Jumanji (1995)',
 'user_age': 30.0,
 'user_id': b'42',
 'user_occupation': b'administrator',
 'user_rating': 4.0,
 'user_zipcode': b'17870'}
{'movie_title': b'While You Were Sleeping (1995)',
 'user_age': 30.0,
 'user_id': b'42',
 'user_occupation': b'administrator',
 'user_rating': 4.0,
 'user_zipcode': b'17870'}
{'movie_title': b'Two if by Sea (1996)',
 'user_age': 30.0,
 'user_id': b'42',
 'user_occupation': b'administrator',
 'user_rating': 3.0,
 'user_zipcode': b'17870'}
{'movie_title': b'Indiana Jones and the Last Crusade (1989)',
 'user_age': 30.0,
 'user_id': b'42',
 'user_occupation': b'administrator',
 'user_rating': 5.0,
 'user_z

In [60]:
RankingModel()((["42"],["administrator"],["17870"],[30.0],["One Flew Over the Cuckoo's Nest (1975)"]))

Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.02617407]], dtype=float32)>

In [61]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [62]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"],features["user_occupation"], features["user_zipcode"],features["user_age"],features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [63]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

model.fit(cached_train, epochs=3,verbose=0)

model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 17ms/step - root_mean_squared_error: 1.1182 - loss: 1.2454 - regularization_loss: 0.0000e+00 - total_loss: 1.2454


{'root_mean_squared_error': 1.1181538105010986,
 'loss': 1.2242404222488403,
 'regularization_loss': 0,
 'total_loss': 1.2242404222488403}

In [64]:
test_ratings = {}
test_movie_titles = ["M*A*S*H (1970)", "Dances with Wolves (1990)", "Speed (1994)","Clear and Present Danger (1994)"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": np.array(["42"]),
      "user_occupation": np.array(["administrator"]),
      "user_zipcode": np.array(["17870"]),
      "user_age": np.array([30.0]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
Clear and Present Danger (1994): [[3.7346609]]
Dances with Wolves (1990): [[3.7111335]]
M*A*S*H (1970): [[3.6967728]]
Speed (1994): [[3.6963124]]
